## Takeaways
- Working out a way to track daily to previous day changes - WIP
- This will need to be seperate from the historical captured data
- CONFIRMED 11/14/2022: Results do not pull current day trading window

## App Testing

In [85]:
# Import Libraries
import pandas as pd
import yfinance as yf
import streamlit as st
import os
import datetime
from pandas.tseries.offsets import BDay

pwd = os.getcwd()

In [105]:
# Import Libraries
import yfinance as yf
import streamlit as st

def ticker_history(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    return ticker_history

def ticker_price_action(ticker_symbol):
    
    ticker_price_action = ticker_history(ticker_symbol)

    # Number of rows displayed
    rows_returned = 1

    # Create derived columns
    ticker_price_action["Ticker"] = ticker_symbol
    ticker_price_action["Wkly_Avg_Close"] = ticker_price_action["Close"].rolling(7).mean()
    ticker_price_action["Wkly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Wkly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Monthly_Avg_Close"] = ticker_price_action["Close"].rolling(30).mean()
    ticker_price_action["Monthly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Monthly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Semi_Annual_Avg_Close"] = ticker_price_action["Close"].rolling(183).mean()
    ticker_price_action["Semi_Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Semi_Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Annual_Avg_Close"] = ticker_price_action["Close"].rolling(365).mean()
    ticker_price_action["Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Two_Year_Avg_Close"] = ticker_price_action["Close"].rolling(730).mean()
    ticker_price_action["Two_Year_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Two_Year_Avg_Close"]) / ticker_price_action["Close"] * 100

    # Drop NAs
    ticker_price_action = ticker_price_action.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_price_action[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    # ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

ticker_price_action("AAPL")


,Date,Ticker,Close,Wkly_Avg_Close_Percent,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close_Percent,Two_Year_Avg_Close_Percent,Wkly_Avg_Close,Monthly_Avg_Close,Semi_Annual_Avg_Close,Annual_Avg_Close,Two_Year_Avg_Close
0,2022-11-15,AAPL,152.059998,5.093854,4.980821,-1.032034,-0.878366,15.570035,144.314283,144.486161,153.629309,153.395641,128.384203


In [84]:
import pandas as pd
from history_data import ticker_price_action
from daily_data import ticker_info
import os
import glob

pwd = os.getcwd()


# Assign function call to ticker for History
mcd_ticker = ticker_price_action("MCD")
pep_ticker = ticker_price_action("PEP")
msft_ticker = ticker_price_action("MSFT")
aapl_ticker = ticker_price_action("AAPL")
o_ticker = ticker_price_action("O")

# Call Daily function
daily_info = ticker_info()

# Export History Data to CSV
mcd_ticker.to_csv(pwd + "\\history.csv", mode ="w", header=True, index=False)
pep_ticker.to_csv(pwd + "\\history.csv", mode ="a", header=False, index=False)
msft_ticker.to_csv(pwd + "\\history.csv", mode ="a", header=False, index=False)
aapl_ticker.to_csv(pwd + "\\history.csv", mode ="a", header=False, index=False)
o_ticker.to_csv(pwd + "\\history.csv", mode ="a", header=False, index=False)

# Export Daily Data to CSV
daily_info.to_csv(pwd + "\\daily.csv", mode ="w", header=True, index=False)

# Create CSV data frame the import into Streamlit
df_hist = pd.read_csv(pwd + "\\history.csv")
df_daily = pd.read_csv(pwd + "\\daily.csv")

df_merged = pd.concat([df_hist, df_daily], ignore_index=True, sort=False)
df_merged

c:\Users\jisbe\pyproject\financial-portfolio\daily_data.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Fifty_Day_Avg_%"] = (df["currentPrice"] - df["fiftyDayAverage"]) / df["currentPrice"] * 100
c:\Users\jisbe\pyproject\financial-portfolio\daily_data.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TwoHundred_Day_Avg_%"] = (df["currentPrice"] - df["twoHundredDayAverage"]) / df["currentPrice"] * 100
c:\Users\jisbe\pyproject\financial-portfolio\daily_data.py:25: PerformanceWarning: DataFrame is highly fragmented.

,Date,Ticker,Close,Wkly_Avg_Close_Percent,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close_Percent,Two_Year_Avg_Close_Percent,Wkly_Avg_Close,Monthly_Avg_Close,...,Fifty_Day_Avg_%,TwoHundred_Day_Avg_%,Week_52_Low_Avg_%,Week_52_High_Avg_%,Previous_Date,previousClose,fiftyDayAverage,twoHundredDayAverage,fiftyTwoWeekLow,fiftyTwoWeekHigh
0,2022-11-14,MCD,272.269989,-1.095031,5.507764,8.735198,10.025939,18.749958,275.251430,257.274000,...,6.675800,7.997190,20.049950,-3.452455,2022-11-11,$271.39,$254.09,$250.50,$217.68,$281.67
1,2022-11-14,PEP,176.729996,-1.424291,1.337065,4.590457,8.112967,17.352337,179.247144,174.367001,...,2.631585,3.868387,13.217903,-3.875969,2022-11-11,$178.05,$172.08,$169.89,$153.37,$183.58
2,2022-11-14,MSFT,241.550003,3.345657,2.934107,-9.910973,-17.029550,-0.445377,233.468569,234.462666,...,0.972304,-11.343117,11.641482,-44.760919,2022-11-11,$247.11,$239.20,$268.95,$213.43,$349.67
3,2022-11-14,AAPL,148.279999,3.992447,2.697983,-3.644231,-3.399628,13.493940,142.359998,144.279429,...,0.517265,-4.916752,12.975452,-23.374697,2022-11-11,$149.70,$147.51,$155.57,$129.04,$182.94
4,2022-11-14,O,64.080002,0.312111,6.113594,-2.958793,-2.222978,5.677123,63.880001,60.162411,...,4.020599,-4.743602,13.389513,-17.665418,2022-11-11,$64.98,$61.50,$67.12,$55.50,$75.40


Working out a way to track daily to previous day changes - WIP

In [ ]:
def ticker_info(): # ticker_symbol

    l = ["PEP", "MCD", "MSFT", "AAPL"]

    df_list = []

    for t in l:
        df_list.append(pd.DataFrame([yf.Ticker(t).info]))
    
    df = pd.concat(df_list)

    return df

df = ticker_info()

df["Current_Date"] = datetime.datetime.today() - BDay(0)
df["Previous_Date"] = datetime.datetime.today() - BDay(1)

# df = df[["symbol"
#         "currentPrice", "previousClose",             
#         "Current_Date", "Previous_Date",
#         "targetLowPrice", "targetHighPrice"
#         # Not sure about needing these columns: 
#         # "fiftyDayAverage", "twoHundredDayAverage", "targetLowPrice", "fiftyTwoWeekLow","targetHighPrice",  "fiftyTwoWeekHigh",
#         # "recommendationKey", "payoutRatio", "lastDividendValue", 
#         # "trailingAnnualDividendYield", "fiveYearAvgDividendYield", "longName", "shortName",
#         # "dividendRate", "dividendYield"
# ]]

# Create derived columns
df["Daily_Percent_Change"] = (df["currentPrice"] - df["previousClose"]) / df["currentPrice"] * 100
df["dividendYield"] = df["dividendYield"] * 100

# Adding percent or dollar symbols
df["currentPrice"] = df["currentPrice"].map("${:,.2f}".format)
df["previousClose"] = df["previousClose"].map("${:,.2f}".format)
df["Daily_Percent_Change"] = df["Daily_Percent_Change"].map("{:,.2f}%".format)
df["dividendRate"] = df["dividendRate"].map("${:,.2f}".format)
df["dividendYield"] = df["dividendYield"].map("{:,.2f}%".format)

# Reorder columns in df

df = df[["symbol"
        "Current_Date", "Previous_Date",
        "currentPrice", "previousClose",             
]]

df

In [80]:
def ticker_info(): # ticker_symbol

    l = ["PEP", "MCD", "MSFT", "AAPL"]

    df_list = []

    for t in l:
        df_list.append(pd.DataFrame([yf.Ticker(t).info]))
    
    df = pd.concat(df_list)

    df["Current_Date"] = datetime.datetime.today() - BDay(0)
    df["Previous_Date"] = datetime.datetime.today() - BDay(1)

    df = df[["symbol", "shortName",
            "currentPrice", "previousClose",      
            "dividendRate", "dividendYield",
            "Current_Date", "Previous_Date",

    ]]

    # Create derived columns
    df["Daily_Percent_Change"] = (df["currentPrice"] - df["previousClose"]) / df["currentPrice"] * 100
    df["dividendYield"] = df["dividendYield"] * 100

    # Adding percent or dollar symbols
    df["currentPrice"] = df["currentPrice"].map("${:,.2f}".format)
    df["previousClose"] = df["previousClose"].map("${:,.2f}".format)
    df["Daily_Percent_Change"] = df["Daily_Percent_Change"].map("{:,.2f}%".format)
    df["dividendRate"] = df["dividendRate"].map("${:,.2f}".format)
    df["dividendYield"] = df["dividendYield"].map("{:,.2f}%".format)

    # Reorder columns in df

    df = df[["symbol", "shortName",
            "currentPrice", "previousClose", "Daily_Percent_Change",
            "dividendRate", "dividendYield",
            "Current_Date", "Previous_Date",
    ]]

    return df

df = ticker_info()



df

,symbol,shortName,currentPrice,previousClose,Daily_Percent_Change,dividendRate,dividendYield,Current_Date,Previous_Date
0,PEP,"Pepsico, Inc.",$177.74,$178.05,-0.17%,$4.60,2.58%,2022-11-14 15:12:47.865427,2022-11-11 15:12:47.866455
0,MCD,McDonald's Corporation,$273.83,$271.39,0.89%,$6.08,2.24%,2022-11-14 15:12:47.865427,2022-11-11 15:12:47.866455
0,MSFT,Microsoft Corporation,$243.01,$247.11,-1.69%,$2.72,1.10%,2022-11-14 15:12:47.865427,2022-11-11 15:12:47.866455
0,AAPL,Apple Inc.,$149.43,$149.70,-0.18%,$0.92,0.61%,2022-11-14 15:12:47.865427,2022-11-11 15:12:47.866455


In [54]:
def ticker_history(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    return ticker_history

In [49]:
def ticker_price_action(ticker_symbol):
    
    ticker_price_action = ticker_history(ticker_symbol)

    # Number of rows displayed
    rows_returned = 5

    # Create derived columns
    ticker_price_action["Ticker"] = ticker_symbol
    ticker_price_action["Wkly_Avg_Close"] = ticker_price_action["Close"].rolling(7).mean()
    ticker_price_action["Wkly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Wkly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Monthly_Avg_Close"] = ticker_price_action["Close"].rolling(30).mean()
    ticker_price_action["Monthly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Monthly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Semi_Annual_Avg_Close"] = ticker_price_action["Close"].rolling(183).mean()
    ticker_price_action["Semi_Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Semi_Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Annual_Avg_Close"] = ticker_price_action["Close"].rolling(365).mean()
    ticker_price_action["Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Two_Year_Avg_Close"] = ticker_price_action["Close"].rolling(730).mean()
    ticker_price_action["Two_Year_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Two_Year_Avg_Close"]) / ticker_price_action["Close"] * 100

    # Drop NAs
    ticker_price_action = ticker_price_action.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_price_action[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

ticker_price_action("PEP")

,Date,Ticker,Close,CurrentPrice,PreviousClose,Daily_Change,Daily_Change_Percent,Wkly_Avg_Close_Percent,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close_Percent,Two_Year_Avg_Close_Percent,Wkly_Avg_Close,Monthly_Avg_Close,Semi_Annual_Avg_Close,Annual_Avg_Close,Two_Year_Avg_Close
0,2022-11-11,PEP,178.050003,178.05,181.7,-0.000003,-0.000002,-0.756609,2.283442,5.338830,8.846193,18.002853,179.397145,173.984334,168.544216,162.299356,145.995922
1,2022-11-10,PEP,181.699997,178.05,181.7,-3.649997,-2.049984,1.252454,4.517701,7.279675,10.730490,19.689054,179.424288,173.491334,168.472828,162.202696,145.924986
2,2022-11-09,PEP,178.550003,178.05,181.7,-0.500003,-0.280822,-0.414449,3.114908,5.696968,9.216106,18.315086,179.290002,172.988334,168.378067,162.094646,145.848416
3,2022-11-08,PEP,180.710007,178.05,181.7,-2.660007,-1.493966,0.546261,4.455208,6.869918,10.357109,19.331299,179.722859,172.659001,168.295378,161.993675,145.776415
4,2022-11-07,PEP,180.210007,178.05,181.7,-2.160007,-1.213146,0.149829,4.462020,6.665296,10.167252,19.148450,179.940000,172.169000,168.198476,161.887600,145.702583


In [ ]:
# Streamlit conditional function
def percent_variance(val):
    color = "red" if val < 0 else "green" # if val > 0 else "green"
    return f"background-color: {color}"

In [10]:
def ticker_financials(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_financials = ticker.quarterly_financials

    # Number of rows displayed
    rows_returned = 5
    
    return ticker_financials

ticker_financials("MCD")

,2022-09-30,2022-06-30,2022-03-31,2021-12-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,2536200000.0,1409100000.0,1541200000.0,2107800000.0
Minority Interest,None,None,None,None
Net Income,1981600000.0,1188000000.0,1104400000.0,1638800000.0
Selling General Administrative,576400000.0,611200000.0,584300000.0,755400000.0
Gross Profit,3446100000.0,3302100000.0,3050100000.0,3261000000.0
Ebit,2776400000.0,2597900000.0,2312600000.0,2419500000.0
Operating Income,2776400000.0,2597900000.0,2312600000.0,2419500000.0
Other Operating Expenses,NaN,NaN,60500000.0,-400000.0


## Execute staging data

In [ ]:
# Assign function call to ticker
mcd_ticker = ticker_price_action("MCD")
pep_ticker = ticker_price_action("PEP")
msft_ticker = ticker_price_action("MSFT")
o_ticker = ticker_price_action("O")

# Export to CSV
mcd_ticker.to_csv(pwd + "\\mcd_stocks.csv", mode ="w", header=True, index=False)
pep_ticker.to_csv(pwd + "\\pep_stocks.csv", mode ="w", header=True, index=False)
msft_ticker.to_csv(pwd + "\\msft_stocks.csv", mode ="w", header=True, index=False)
o_ticker.to_csv(pwd + "\\o_stocks.csv", mode ="w", header=True, index=False)

# Create CSV data frame the import into Streamlit
mcd_df = pd.read_csv(pwd + "\\mcd_stocks.csv")
pep_df = pd.read_csv(pwd + "\\pep_stocks.csv")
msft_df = pd.read_csv(pwd + "\\msft_stocks.csv")
o_df = pd.read_csv(pwd + "\\o_stocks.csv")

## Previous script versions

### Iteration 2: Consolidated into functions, but very high coupling and low cohesion (bad)

In [5]:
def ticker_function(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    # Number of rows displayed
    rows_returned = 5

    # Create derived columns
    ticker_history["Ticker"] = ticker_symbol
    ticker_history["Wkly_Avg_Close"] = ticker_history["Close"].rolling(7).mean()
    ticker_history["Wkly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Wkly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Monthly_Avg_Close"] = ticker_history["Close"].rolling(30).mean()
    ticker_history["Monthly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Monthly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Semi_Annual_Avg_Close"] = ticker_history["Close"].rolling(183).mean()
    ticker_history["Semi_Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Semi_Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Annual_Avg_Close"] = ticker_history["Close"].rolling(365).mean()
    ticker_history["Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Two_Year_Avg_Close"] = ticker_history["Close"].rolling(730).mean()
    ticker_history["Two_Year_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Two_Year_Avg_Close"]) / ticker_history["Close"] * 100

    # Drop NAs
    ticker_history = ticker_history.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_history[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

# Assign function call to ticker
# mcd_ticker = ticker_function("MCD")
# pep_ticker = ticker_function("PEP")
# msft_ticker = ticker_function("MSFT")

# Export to CSV
# mcd_ticker.to_csv(pwd + "\\mcd_stocks.csv", mode ="w", header=True, index=False)
# pep_ticker.to_csv(pwd + "\\pep_stocks.csv", mode ="w", header=True, index=False)
# msft_ticker.to_csv(pwd + "\\msft_stocks.csv", mode ="w", header=True, index=False)

# Create CSV data frame the import into Streamlit
# df = pd.read_csv(pwd + "\\stocks.csv")

# Run Streamlit
# st.title("Weekly DCA Assessment")
# st.write(df)

# add_sidebar = st.sidebar.selectbox("Test Side Bar Title", ("Test 1", "Test 2"))



### Iteration 1: Not yet consolidated into functions

In [105]:
mcd = yf.Ticker("mcd")
mcd_history = mcd.history(period="max")

# Create Ticker Column
mcd_history["Ticker"] = "MCD"

# Create rolling weekly average column
mcd_history["weekly_avg_close"] = mcd_history["Close"].rolling(7).mean()

# Create weekly average close percent column
mcd_history["weekly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["weekly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling monthly average column
mcd_history["monthly_avg_close"] = mcd_history["Close"].rolling(30).mean()

# Create monthly average close percent column
mcd_history["monthly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["monthly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling semi_annual average column
mcd_history["semi_annual_avg_close"] = mcd_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
mcd_history["semi_annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["semi_annual_avg_close"]) / mcd_history["Close"] * 100

# Create rolling annual average column
mcd_history["annual_avg_close"] = mcd_history["Close"].rolling(365).mean()

# Create annual average close percent column
mcd_history["annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["annual_avg_close"]) / mcd_history["Close"] * 100


# Create rolling two_year average column
mcd_history["two_year_avg_close"] = mcd_history["Close"].rolling(730).mean()

# Create two_year average close percent column
mcd_history["two_year_avg_close_percent"] = (mcd_history["Close"] - mcd_history["two_year_avg_close"]) / mcd_history["Close"] * 100

# Drop NAs
mcd_history = mcd_history.dropna()

# Grab only needed columns
mcd_price_action = mcd_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
mcd_price_action = mcd_price_action.reset_index()

# Export to CSV
# mcd_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [106]:
pep = yf.Ticker("PEP")
pep_history = pep.history(period="max")

# Create Ticker Column
pep_history["Ticker"] = "PEP"

# Create rolling weekly average column
pep_history["weekly_avg_close"] = pep_history["Close"].rolling(7).mean()

# Create weekly average close percent column
pep_history["weekly_avg_close_percent"] = (pep_history["Close"] - pep_history["weekly_avg_close"]) / pep_history["Close"] * 100

# Create rolling monthly average column
pep_history["monthly_avg_close"] = pep_history["Close"].rolling(30).mean()

# Create monthly average close percent column
pep_history["monthly_avg_close_percent"] = (pep_history["Close"] - pep_history["monthly_avg_close"]) / pep_history["Close"] * 100

# Create rolling semi_annual average column
pep_history["semi_annual_avg_close"] = pep_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
pep_history["semi_annual_avg_close_percent"] = (pep_history["Close"] - pep_history["semi_annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling annual average column
pep_history["annual_avg_close"] = pep_history["Close"].rolling(365).mean()

# Create annual average close percent column
pep_history["annual_avg_close_percent"] = (pep_history["Close"] - pep_history["annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling two_year average column
pep_history["two_year_avg_close"] = pep_history["Close"].rolling(730).mean()

# Create two_year average close percent column
pep_history["two_year_avg_close_percent"] = (pep_history["Close"] - pep_history["two_year_avg_close"]) / pep_history["Close"] * 100


# Drop NAs
pep_history = pep_history.dropna()

# Grab only needed columns
pep_price_action = pep_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
pep_price_action = pep_price_action.reset_index()


# Export to CSV
# pep_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [107]:
msft = yf.Ticker("MSFT")
msft_history = msft.history(period="max")

msft_history["Ticker"] = "MSFT"

# Create rolling weekly average column
msft_history["weekly_avg_close"] = msft_history["Close"].rolling(7).mean()

# Create weekly average close percent column
msft_history["weekly_avg_close_percent"] = (msft_history["Close"] - msft_history["weekly_avg_close"]) / msft_history["Close"] * 100

# Create rolling monthly average column
msft_history["monthly_avg_close"] = msft_history["Close"].rolling(30).mean()

# Create monthly average close percent column
msft_history["monthly_avg_close_percent"] = (msft_history["Close"] - msft_history["monthly_avg_close"]) / msft_history["Close"] * 100

# Create rolling semi_annual average column
msft_history["semi_annual_avg_close"] = msft_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
msft_history["semi_annual_avg_close_percent"] = (msft_history["Close"] - msft_history["semi_annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling annual average column
msft_history["annual_avg_close"] = msft_history["Close"].rolling(365).mean()

# Create annual average close percent column
msft_history["annual_avg_close_percent"] = (msft_history["Close"] - msft_history["annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling two_year average column
msft_history["two_year_avg_close"] = msft_history["Close"].rolling(730).mean()

# Create two_year average close percent column
msft_history["two_year_avg_close_percent"] = (msft_history["Close"] - msft_history["two_year_avg_close"]) / msft_history["Close"] * 100


# Drop NAs
msft_history = msft_history.dropna()

# Grab only needed columns
msft_price_action = msft_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
msft_price_action = msft_price_action.reset_index()

# Export to CSV
# msft_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)